In [3]:
import torch
from transformers import AutoTokenizer, BertForMaskedLM
import numpy as np

# 1. BERT

In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
inputs = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
], return_tensors='pt', padding=True)

for tokens in inputs.input_ids:
    token_len = 0
    for token in tokens:
        if token != torch.tensor(0):
            token_len += 1
            continue
        break
    tokens[np.random.randint(1, token_len - 2)] = tokenizer.mask_token_id

with torch.inference_mode():
    logits = model(**inputs).logits

sentence_index, mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)

predicted_token_id = []
for index, sentence in enumerate(sentence_index):
    if sentence >= len(predicted_token_id):
        predicted_token_id.append([])

    predicted_token_id[-1].append(tokenizer.decode(logits[sentence, mask_token_index[index]].argmax(axis=-1)))

In [11]:
tokenizer.batch_decode(inputs.input_ids)

['[CLS] The capital of [MASK] is Paris. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] The capital of France [MASK] Paris. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] The capital [MASK] France is Paris. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 우 [MASK]나라의 수도는 Seoul 이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 강남에는 외국 [MASK] 많이 보인다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Javascript 는 웹개발에 필수적인 programming [MASK]어이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 [MASK] 가자. [SEP]']

In [12]:
print(logits.shape)
print(sentence_index, mask_token_index)
print(predicted_token_id)

torch.Size([7, 27, 119547])
tensor([0, 1, 2, 3, 4, 5, 6]) tensor([ 4,  5,  3,  2,  6, 13, 22])
[['France'], ['is'], ['of'], ['##리'], ['##이'], ['언'], ['함께']]


In [13]:
labels = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
], return_tensors='pt', padding=True)['input_ids']

labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)

In [15]:
print(labels.shape)
print(outputs.loss)

torch.Size([7, 27])
tensor(1.4978, grad_fn=<NllLossBackward0>)


# 2. KoBERT

In [16]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
inputs = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
], return_tensors='pt', padding=True)

for tokens in inputs.input_ids:
    token_len = 0
    for token in tokens:
        if token != torch.tensor(0):
            token_len += 1
            continue
        break
    tokens[np.random.randint(1, token_len - 2)] = tokenizer.mask_token_id

with torch.inference_mode():
    logits = model(**inputs).logits

sentence_index, mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)

predicted_token_id = []
for index, sentence in enumerate(sentence_index):
    if sentence >= len(predicted_token_id):
        predicted_token_id.append([])

    predicted_token_id[-1].append(tokenizer.decode(logits[sentence, mask_token_index[index]].argmax(axis=-1)))

In [18]:
tokenizer.batch_decode(inputs.input_ids)

['[CLS] The capital of France is[MASK]aris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] The c[MASK]pital of France is Paris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] The capital[MASK] France is Paris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] 우리나라의 수도[MASK] Seoul 이다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] 강남에는 외국인이 많이 보인다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][MASK][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] Javascript 는 웹개발에 필수적인 pro[MASK]ramming 언어이다.[SEP]',
 '[CLS] 나 오늘 미용실[MASK] 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.[SEP][PAD][PAD][PAD][PAD]']

In [19]:
print(logits.shape)
print(sentence_index, mask_token_index)
print(predicted_token_id)

torch.Size([7, 28, 8002])
tensor([0, 1, 2, 3, 4, 5, 6]) tensor([16,  4,  8,  4, 16, 19,  6])
[['뻤'], ['총'], ['열애설'], ['깍'], ['불리는'], ['늘'], ['껑']]


In [20]:
labels = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
], return_tensors='pt', padding=True)['input_ids']

labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)

In [22]:
print(labels.shape)
print(outputs.loss)

torch.Size([7, 28])
tensor(11.6062, grad_fn=<NllLossBackward0>)
